In [9]:
import pandas as pd 
from sklearn import tree
fires_origin = pd.read_csv("forestfires.csv", header=0, index_col=False) 
print(fires_origin.head())

   X  Y month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain  area
0  7  5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   0.0
1  7  4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   0.0
2  7  4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0   0.0
3  8  6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2   0.0
4  8  6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0   0.0


In [10]:
temp_months = fires_origin['month'].copy()
temp_days = fires_origin['day'].copy()
print(temp_months.head(), temp_days.head())

0    mar
1    oct
2    oct
3    mar
4    mar
Name: month, dtype: object 0    fri
1    tue
2    sat
3    fri
4    sun
Name: day, dtype: object


In [11]:
d = {
    'X': fires_origin['X'].values, 
    'Y': fires_origin['Y'].values, 
    'temp': fires_origin['temp'].values,
    'RH': fires_origin['RH'].values,
    'wind': fires_origin['wind'].values,
    'rain': fires_origin['rain'].values,
    'area': fires_origin['area'].values
}
fires = pd.DataFrame(d)
print(fires.head())

   X  Y  temp  RH  wind  rain  area
0  7  5   8.2  51   6.7   0.0   0.0
1  7  4  18.0  33   0.9   0.0   0.0
2  7  4  14.6  33   1.3   0.0   0.0
3  8  6   8.3  97   4.0   0.2   0.0
4  8  6  11.4  99   1.8   0.0   0.0


In [12]:
import statistics as stat
maximum = max(fires['temp'])
minimum = min(fires['temp'])
median = stat.median(fires['temp'])
diff = round((maximum - median)/4)

In [13]:
tp = fires['temp']
serious = fires['temp'].copy()
for i, val in enumerate(fires['temp']):
    #print(val, rh[i], serious[i])
    if (val >= (median + diff)):
        serious.loc[i] = 1
    else:
        serious.loc[i] = 0
print(serious[1:20])

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     1.0
7     0.0
8     0.0
9     1.0
10    0.0
11    0.0
12    0.0
13    0.0
14    1.0
15    1.0
16    0.0
17    0.0
18    0.0
19    0.0
Name: temp, dtype: float64


In [24]:
x = pd.DataFrame(fires.values[:,0:13], columns =['X','Y','temp', 'RH', 'wind', 'rain', 'area'])
y = pd.DataFrame(serious.values, columns =['SERIOUS'])

In [25]:
print(x.shape)
print(y.shape)

(517, 7)
(517, 1)


In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.9, random_state=42)

In [30]:
train_class1 = 0;
train_class2 = 0;
for val in y_train['SERIOUS']:
    if (val == 0.0):
        train_class1 = train_class1 + 1
    else:
        if (val == 1.0):
            train_class2 = train_class2 + 1
test_class1 = 0;
test_class2 = 0;
for val in y_test['SERIOUS']:
    if (val == 0.0):
        test_class1 = test_class1 + 1
    else:
        if (val == 1.0):
            train_class2 = train_class2 + 1
print('0.0:', train_class1, '1.0:', train_class2)
print('0.0:', test_class1, '1.0:', test_class2)
# widzimy, że zbiór jest niezbalansowany

0.0: 39 1.0: 141
0.0: 337 1.0: 0


In [18]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True)

In [21]:
from sklearn.model_selection import cross_val_score
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train.values.ravel())
print(cross_val_score(clf, x, y, cv=skf))

[1. 1. 1. 1. 1.]


In [22]:
from sklearn.metrics import accuracy_score
y_test_pred_from_clf = clf.predict(x_test)
print(accuracy_score(y_test, y_test_pred_from_clf))

1.0
